In [258]:
import numpy as np
import pandas as pd
# import sklearn.datasets as data

In [259]:
df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')
df.head()
print(df.shape)


(305434, 45)


Dropped Meldingstidspunkt, Starttidspunkt, Art - FDIR and strings from dataset due to how the data scaling generaly dislikes strings!
After dropping stings the scores went from -11% to 20%! (Both test and training) 

Maybe use one-hot encoding to include it anyway?

In [260]:
#Sile ut informasjon:
df = df[(df['Art - FDIR (kode)'] == 1022) | (df['Art - FDIR (kode)'] == 1027)] # Fiske som fanger hyse eller torsk. "751 - lodde, 1022 Torsk"
area = [7, 8, 28, 30, 40]
df = df[(df['Redskap FDIR (kode)'] == 51) & (df['Hovedområde start (kode)'].isin(area))] # Fiske med bunntrål - 51, snurrevad - 11
df['Bruttotonnasje Kombinert'] = df['Bruttotonnasje 1969'].fillna(df['Bruttotonnasje annen']) # Slår sammen kolonnene.
# Dropper unødvendig informasjon. Tabellen inneholder masse duplikater. som egen kolonne for navn og for kode
# Forskjellige klassifiseringsmåter. Holder på FDIR sine.
df.drop(columns = ['Meldingsdato', 
                   'Meldingsklokkeslett', 
                   'Startdato',  
                   'Hovedområde start', 
                   'Lokasjon start (kode)', 
                   'Stoppdato', 
                   'Stoppklokkeslett', 
                   'Fangstår', 
                   'Hovedområde stopp (kode)', 
                   'Hovedområde stopp', 
                   'Lokasjon stopp (kode)', 
                   'Redskap FAO (kode)', 
                   'Redskap FAO', 
                   'Redskap FDIR', 
                   'Hovedart FAO (kode)', 
                   'Hovedart FAO', 
                   'Art FAO (kode)',
                   'Art FAO',
                   'Art - gruppe', 
                   'Lengdegruppe', 
                   'Bredde', 
                   'Fartøylengde',
                   'Bruttotonnasje 1969',
                   'Bruttotonnasje annen',

                    #Columns removed for training of MLPRegressor neural network
                   'Meldingstidspunkt',
                   'Starttidspunkt',
                   'Startklokkeslett',
                   'Stopptidspunkt',
                   'Art - FDIR',
    ], inplace=True)

# Gruppere bruttotonnasje:
bins = [x for x in range(0, 4800, 100)] # Definerer gruppene
labels = range(1, len(bins))  # nummererer gruppene.
# Categorize 'Bruttotonnasje Kombinert' into the defined bins
df['Bruttotonnasje Gruppert'] = pd.cut(df['Bruttotonnasje Kombinert'], bins=bins, labels=labels)
# Teller grupperingene
df['Bruttotonnasje Gruppert'].value_counts()

Bruttotonnasje Gruppert
6     992
3     950
2     618
21    465
4     433
35    312
14    305
32    261
5     257
25    233
1     210
12    191
16    186
15    181
40    164
38     78
26     50
7      49
23     45
19     44
45     17
17      8
42      5
44      0
46      0
41      0
33      0
37      0
36      0
43      0
34      0
39      0
24      0
31      0
30      0
29      0
28      0
27      0
22      0
20      0
18      0
13      0
11      0
10      0
9       0
8       0
47      0
Name: count, dtype: int64

In [261]:
df.head()

,Melding ID,Startposisjon bredde,Startposisjon lengde,Hovedområde start (kode),Havdybde start,Varighet,Stopposisjon bredde,Stopposisjon lengde,Havdybde stopp,Trekkavstand,Redskap FDIR (kode),Hovedart - FDIR (kode),Art - FDIR (kode),Art - gruppe (kode),Rundvekt,Lengdegruppe (kode),Bruttotonnasje Kombinert,Bruttotonnasje Gruppert
353,1497555,57.588,6.834,8.0,-229,692,57.534,7.466,-237,38303.0,51.0,2524.0,1022.0,201.0,114.0,4.0,267.0,3
635,1497856,61.221,2.181,28.0,-167,330,61.481,1.753,-193,36931.0,51.0,1032.0,1022.0,201.0,100.0,5.0,2053.0,21
1000,1498229,58.318,6.210,8.0,1,71,58.328,6.200,41,1259.0,51.0,2524.0,1022.0,201.0,10.0,3.0,79.0,1
1001,1498229,58.318,6.210,8.0,1,71,58.328,6.200,41,1259.0,51.0,2524.0,1027.0,202.0,8.0,3.0,79.0,1
1149,1498376,63.411,5.656,7.0,-356,302,63.346,5.699,-224,7558.0,51.0,1032.0,1022.0,201.0,72.0,5.0,536.0,6


In [262]:
print(df.shape)
df.head()
X = df.drop('Rundvekt', axis=1)
y = df['Rundvekt']


(6054, 18)


In [263]:
# from sklearn.datasets import make_regression
# x = df.drop('Rundvekt', axis=1)
# y = df['Rundvekt']
# X, y = make_regression(n_samples=200, random_state=1)
# # print(X,y)

In [264]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)
# print(X)

In [265]:
# X.shape = (6054, 23)
# y.shape = (6054,)

In [266]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 1235)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [267]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

scaler = sc.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_testscaled = scaler.transform(X_test)

In [268]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor(hidden_layer_sizes = [20,10], 
                  activation = 'logistic', alpha = 0.1, 
                  max_iter = 1000, #How many iterations of regression you do. higher = bigger chance of overtraining
                  solver = 'adam', 
                  verbose = False)
#activation logistic tanh relu identity
#solver adam lbfgs sgd 

In [269]:
nn.fit(X_train,y_train)
print("Neural network, MLPRegressor")
print("Test score",  nn.score(X_test, y_test))
print("Train  score", nn.score(X_train, y_train))

Neural network, MLPRegressor
Test score 0.02857554746080171
Train  score 0.04388748121910424


c:\Mashinelearning seminaroppgave\INFO284\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


V2, removed linear regression thingy
Max_iter = 5000
Test score 0.18474766635857465
Train  score 0.26563262873718185

Without removed collumns
Test score 0.03306077219837722
Train  score 0.07260871122128132

with removed collumnds (linear regression collumns)
Test score 0.034291607331412455
Train  score 0.07651381759634102